In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
pip install nltk


In [ ]:
pip install --upgrade openai


In [ ]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

# Load the dataset
file_path = '/content/newalternatedataset.csv'  # Update this path if needed
df = pd.read_csv(file_path)

# Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

# Apply preprocessing to the text column
df['processed_text'] = df['text'].apply(preprocess_text)

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models like 'all-MiniLM-L12-v2'

# Generate embeddings for each row and add to DataFrame
df['embedding'] = df['processed_text'].apply(lambda x: model.encode(x, convert_to_tensor=True).tolist())

# Drop the original text and processed_text columns
df = df.drop(columns=['text', 'processed_text'])

# Save to a new CSV file
df.to_csv('/content/embedded_sbert.csv', index=False)

print("Embedding complete and saved as 'embedded_sbert.csv'")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding complete and saved as 'embedded_sbert.csv'


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, RepeatVector, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter
from sklearn.metrics import accuracy_score

In [ ]:
# Load your dataset
data = pd.read_csv('embedded_sbert.csv')

# Extract features and labels
X_embedded = np.array(data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=',')).tolist())  # Convert embedded_text to array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values  # Labels


In [ ]:
# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)  # Presence or absence

In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_embedded, y_binary, test_size=0.2, random_state=42)

In [ ]:
# Function to calculate inverse frequency weights for each class
def compute_inverse_freq_weights(y):
    class_weights = {}
    for i in range(y.shape[1]):  # Loop over each label
        label_counts = Counter(y[:, i])  # Count occurrences of 0 and 1 in each label
        total = len(y)  # Total number of samples
        # Calculate weight for each class as inverse of its frequency
        class_weights[i] = {
            0: total / (2 * label_counts[0]),  # Weight for class 0 (negative)
            1: total / (2 * label_counts[1])   # Weight for class 1 (positive)
        }
    return class_weights

# Compute inverse frequency weights for the training labels
class_weights = compute_inverse_freq_weights(y_binary)


In [ ]:
# Reshape the data for CNN input
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)


In [ ]:
# Convert the class weights into a tensor for use in the loss function
class_weights_tensor = np.array([[class_weights[i][0], class_weights[i][1]] for i in range(output_dim)], dtype=np.float32)
class_weights_tensor = tf.constant(class_weights_tensor)

# Define custom weighted binary crossentropy loss function
# Define custom weighted binary crossentropy loss function
def weighted_binary_crossentropy(y_true, y_pred):
    # Calculate the binary crossentropy loss
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Reshape bce_loss to add a dimension for the output classes
    bce_loss = tf.expand_dims(bce_loss, axis=-1) # Add dimension to match weights

    # Compute the weights for the true labels (0 or 1)
    # using broadcasting for efficient computation
    weights = tf.where(tf.equal(y_true, 1),
                       class_weights_tensor[..., 1],  # Weight for class 1 (positive)
                       class_weights_tensor[..., 0])  # Weight for class 0 (negative)


    # Apply the weights to the binary crossentropy loss
    weighted_loss = bce_loss * weights

    # Return the mean weighted loss across samples and time steps
    return tf.reduce_mean(weighted_loss)



NameError: name 'output_dim' is not defined

In [ ]:
# Convert the class weights into a tensor for use in the loss function
# Get the output dimension from the class_weights dictionary
output_dim = len(class_weights) # Assuming each key in class_weights represents an output dimension

class_weights_tensor = np.array([[class_weights[i][0], class_weights[i][1]] for i in range(output_dim)], dtype=np.float32)
class_weights_tensor = tf.constant(class_weights_tensor)

# Define custom weighted binary crossentropy loss function
# Define custom weighted binary crossentropy loss function
def weighted_binary_crossentropy(y_true, y_pred):
    # Calculate the binary crossentropy loss
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Reshape bce_loss to add a dimension for the output classes
    bce_loss = tf.expand_dims(bce_loss, axis=-1) # Add dimension to match weights

    # Compute the weights for the true labels (0 or 1)
    # using broadcasting for efficient computation
    weights = tf.where(tf.equal(y_true, 1),
                       class_weights_tensor[..., 1],  # Weight for class 1 (positive)
                       class_weights_tensor[..., 0])  # Weight for class 0 (negative)


    # Apply the weights to the binary crossentropy loss
    weighted_loss = bce_loss * weights

    # Return the mean weighted loss across samples and time steps
    return tf.reduce_mean(weighted_loss)

In [ ]:
# Reshape y for the decoder to match the output
num_time_steps = X_train.shape[1]  # This should match the sequence length in your data
y_train_seq = np.repeat(y_train[:, np.newaxis, :], num_time_steps, axis=1)
y_val_seq = np.repeat(y_val[:, np.newaxis, :], num_time_steps, axis=1)

# Ensure the types are float32
y_train_seq = y_train_seq.astype(np.float32)
y_val_seq = y_val_seq.astype(np.float32)


In [ ]:
# Model creation
def create_seq2seq_cnn_model(input_shape, output_dim):
    # Encoder
    encoder_inputs = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(encoder_inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=64, kernel_size=2, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # Flatten the output before Dense
    x = Flatten()(x)
    encoder_outputs = Dense(128, activation='relu')(x)

    # Expand dimensions to match the expected input for LSTM
    encoder_outputs = RepeatVector(num_time_steps)(encoder_outputs)

    # Decoder
    decoder_lstm = LSTM(32, return_sequences=True)(encoder_outputs)
    decoder_outputs = TimeDistributed(Dense(output_dim, activation='sigmoid'))(decoder_lstm)

    # Create the model
    model = Model(encoder_inputs, decoder_outputs)
    return model

# Initialize and compile the model
input_shape = (X_embedded.shape[1], 1)  # Input shape
output_dim = y_binary.shape[1]  # Output shape
model = create_seq2seq_cnn_model(input_shape, output_dim)
model.compile(optimizer='adam', loss=weighted_binary_crossentropy, metrics=['accuracy'])


In [ ]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train_seq,
                    batch_size=32,
                    epochs=10,
                    validation_data=(X_val, y_val_seq),
                    callbacks=[early_stopping])


Epoch 1/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 94s 51ms/step - accuracy: 0.3376 - loss: 0.6274 - val_accuracy: 0.9116 - val_loss: 0.5912
Epoch 2/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 69s 50ms/step - accuracy: 0.6812 - loss: 0.5032 - val_accuracy: 0.9083 - val_loss: 0.6043
Epoch 3/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 81s 50ms/step - accuracy: 0.7043 - loss: 0.4522 - val_accuracy: 0.5570 - val_loss: 0.5704
Epoch 4/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 80s 48ms/step - accuracy: 0.5189 - loss: 0.4215 - val_accuracy: 0.3434 - val_loss: 0.5954
Epoch 5/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 84s 50ms/step - accuracy: 0.4205 - loss: 0.3698 - val_accuracy: 0.6619 - val_loss: 0.6790
Epoch 6/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 68s 50ms/step - accuracy: 0.4569 - loss: 0.3681 - val_accuracy: 0.3950 - val_loss: 0.6866
Epoch 7/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 83s 51ms/step - accuracy: 0.5513 - loss: 0.3236 - val_accuracy: 0.3985 - val_loss: 0.7832
Epoch 8/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 80s 50ms/step - accuracy: 0.5557 -

In [ ]:

# Predictions on the validation set
y_pred_probs = model.predict(X_val)

# Set a threshold for binary classification based on probabilities
threshold = 0.5
y_pred = (y_pred_probs[:, -1, :] > threshold).astype(int)  # Get predictions from the last time step




344/344 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step


In [ ]:
# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy by comparing the exact match of all labels
overall_accuracy = accuracy_score(y_val, y_pred)

# Print the overall accuracy
print(f"\nOverall Accuracy: {overall_accuracy:.4f}")

Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       1.00      0.88      0.93     10887
   Provoking Violence       0.94      0.58      0.72      9012
Individual Harassment       1.00      0.86      0.92     10906

            micro avg       0.98      0.79      0.87     30805
            macro avg       0.98      0.77      0.86     30805
         weighted avg       0.98      0.79      0.87     30805
          samples avg       0.86      0.78      0.81     30805


Overall Accuracy: 0.5699


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Display sample examples from the validation set
def display_examples(X, y_true, y_pred, num_examples=25):
    for i in range(num_examples):
        print(f"Example {i+1}:")
        print(f"True Labels (y): {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display sample examples from the validation set
display_examples(X_val, y_val, y_pred)


Example 1:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 2:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 3:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 4:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 5:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 6:
True Labels (y): [1 1 1]
Predicted Labels: [0 0 0]

Example 7:
True Labels (y): [1 0 1]
Predicted Labels: [0 0 1]

Example 8:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 9:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 10:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 11:
True Labels (y): [1 1 1]
Predicted Labels: [0 0 0]

Example 12:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 13:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 14:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 15:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 16:
True Labels (y): [1 1 1]
Predicted La

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import AdamW

# Load the dataset
data = pd.read_csv('embedded_sbert.csv')

# Convert the 'embedded_text' to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Create a custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define a transformer-based model for multilabel classification
class TransformerMultilabelClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformerMultilabelClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Increase input dimension for better learning
        self.dropout = nn.Dropout(0.3)  # Dropout layer to prevent overfitting
        self.fc2 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for multilabel output
        return x

# Instantiate the model
input_dim = X.shape[1]  # Number of features from BERT embeddings
output_dim = y_binary.shape[1]  # Number of labels
model = TransformerMultilabelClassifier(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for multilabel
optimizer = AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            predictions.append(outputs.cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.1608
Epoch 2/10, Loss: 0.1492
Epoch 3/10, Loss: 0.1469
Epoch 4/10, Loss: 0.1447
Epoch 5/10, Loss: 0.1423
Epoch 6/10, Loss: 0.1400
Epoch 7/10, Loss: 0.1382
Epoch 8/10, Loss: 0.1357
Epoch 9/10, Loss: 0.1335
Epoch 10/10, Loss: 0.1308
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.84      0.96      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.94      0.99      0.97     30805
            macro avg       0.94      0.99      0.96     30805
         weighted avg       0.95      0.99      0.97     30805
          samples avg       0.95      0.98      0.96     30805

Overall Accuracy: 0.8221


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch

# Set the device to GPU if available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the correct device
model.to(device)

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=25):
    # Ensure that y_pred is in binary format
    y_pred = (X > 0.4).astype(int)  # Apply threshold to get binary predictions

    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels (y): {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Prepare the input tensor and move it to the correct device
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)

# Set model to evaluation mode and get predictions
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    y_pred = model(X_val_tensor)  # Get model predictions

# Convert predictions to binary format
y_pred_binary = (y_pred.cpu().numpy() > 0.5).astype(int)  # Move predictions back to CPU and threshold

# Now, display the examples
display_examples(y_pred_binary, y_val, num_examples=25)



Example 1:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 2:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 3:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 4:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 5:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 6:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 7:
True Labels (y): [1 0 1]
Predicted Labels: [1 0 1]

Example 8:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 9:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 10:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 11:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 12:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 13:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 14:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 15:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 16:
True Labels (y): [1 1 1]
Predicted La

In [ ]:
pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load the dataset
data = pd.read_csv('embedded_sbert.csv')

# Convert the 'embedding' column to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Define an XGBoost DMatrix for GPU
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set up XGBoost parameters for GPU usage
params = {
    'objective': 'binary:logistic',  # Binary classification for each label
    'tree_method': 'gpu_hist',       # Use GPU acceleration
    'eval_metric': 'logloss',        # Log loss for binary classification
    'predictor': 'gpu_predictor',
    'learning_rate': 0.01,
    'max_depth': 6,
    'verbosity': 1
}

# Train one model per label (multi-label setup)
models = []
for i in range(y_train.shape[1]):
    print(f"Training model for label {i+1}/{y_train.shape[1]}")
    # Train the model for each label
    model = xgb.train(params, dtrain, num_boost_round=100)
    models.append(model)

# Predict and evaluate
y_pred = np.zeros(y_val.shape)
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load the dataset
data = pd.read_csv('embedded_sbert.csv')

# Convert the 'embedding' column to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Define an XGBoost DMatrix for GPU
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set up XGBoost parameters for GPU usage
params = {
    'objective': 'binary:logistic',  # Binary classification for each label
    'tree_method': 'gpu_hist',       # Use GPU acceleration
    'eval_metric': 'logloss',        # Log loss for binary classification
    'predictor': 'gpu_predictor',
    'learning_rate': 0.01,
    'max_depth': 6,
    'verbosity': 1
}

# Train one model per label (multi-label setup)
models = []
for i in range(y_train.shape[1]):
    print(f"Training model for label {i+1}/{y_train.shape[1]}")
    # Train the model for each label
    model = xgb.train(params, dtrain, num_boost_round=100)
    models.append(model)

# Predict and evaluate
y_pred = np.zeros(y_val.shape)
for i, model in enumerate(models):
    # Predict probabilities for the current label only
    y_pred[:, i] = model.predict(dval)[:, i]

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {X[i]}\n")

# Display examples from the validation set
display_examples(y_pred_binary, y_val, num_examples=10)


# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {X[i]}\n")

# Display examples from the validation set
display_examples(y_pred_binary, y_val, num_examples=10)


Training model for label 1/3


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:29:58] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:29:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Training model for label 2/3


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Training model for label 3/3


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:25] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Training model for label 3/3


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.82      1.00      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.93      1.00      0.97     30805
            macro avg       0.93      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.93      0.99      0.96     30805

Overall Accuracy: 0.8201
Example 1:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 2:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 3:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 4:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 5:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 6:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 7:
True Labels: [1 0 1]
Predicted Labels: [1 1 1]

Example 8:
True Labels: [1 1 1]


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:30:32] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('embedded_sbert.csv')

# Convert 'embedding' to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
batch_size = 16  # Define batch size
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Adjusted Hybrid CNN-Attention model without LSTM
class HybridMultilabelClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HybridMultilabelClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(2)

        # Fully connected layer after flattening
        self.fc1 = nn.Linear(128 * (input_dim // 2), 64)

        # Attention mechanism
        self.attention = nn.Linear(64, 1)

        # Final fully connected layer for classification
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension for Conv1d
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))

        # Attention mechanism
        attention_weights = torch.softmax(self.attention(x), dim=1)
        x = attention_weights * x  # Apply attention
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for multilabel output
        return x

# Instantiate model, loss function, optimizer
input_dim = X.shape[1]  # Number of features in embeddings
output_dim = y_binary.shape[1]  # Number of labels

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridMultilabelClassifier(input_dim=input_dim, output_dim=output_dim).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for multilabel
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].cpu().numpy()

            outputs = model(input_ids).cpu().numpy()
            predictions.append(outputs)
            true_labels.append(labels)

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Example predictions on validation set
def example_predictions(model, val_loader, n_examples=5):
    model.eval()
    examples = []
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i == n_examples:
                break
            input_ids = batch['input_ids'].to(device)
            outputs = model(input_ids).cpu().numpy()
            predictions = (outputs > 0.5).astype(int)
            examples.append(predictions)

    return examples

# Print example predictions
example_preds = example_predictions(model, val_loader)
for i, preds in enumerate(example_preds, 1):
    print(f"Example {i}: Predictions - {preds}")


Epoch 1/10, Loss: 0.1644
Epoch 2/10, Loss: 0.1506
Epoch 3/10, Loss: 0.1470
Epoch 4/10, Loss: 0.1448
Epoch 5/10, Loss: 0.1429
Epoch 6/10, Loss: 0.1410
Epoch 7/10, Loss: 0.1389
Epoch 8/10, Loss: 0.1365
Epoch 9/10, Loss: 0.1343
Epoch 10/10, Loss: 0.1316
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.86      0.93      0.89      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.95      0.98      0.96     30805
            macro avg       0.95      0.98      0.96     30805
         weighted avg       0.95      0.98      0.97     30805
          samples avg       0.95      0.97      0.96     30805

Overall Accuracy: 0.8127
Example 1: Predictions - [[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 0 1]
 [1 0 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
Example 2: Predictions - [[1 1 1]
 [1 1 1]
 [1 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('/embedded_sbert.csv')

# Convert 'embedded_text' to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Hybrid Model with CNN, BiLSTM, and Separate Attention Layers
class HybridCNNBiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HybridCNNBiLSTMClassifier, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)

        # BiLSTM layer
        self.bilstm = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)

        # Shared fully connected layer
        self.fc_shared = nn.Linear(128 * 2 * (input_dim // 2), 128)  # Adjusted for BiLSTM output

        # Separate attention and classification layers for each label
        self.attention_emotional = nn.Linear(128, 1)
        self.fc_emotional = nn.Linear(128, 1)

        self.attention_violence = nn.Linear(128, 1)
        self.fc_violence = nn.Linear(128, 1)

        self.attention_harassment = nn.Linear(128, 1)
        self.fc_harassment = nn.Linear(128, 1)

    def forward(self, x):
        # CNN
        x = x.unsqueeze(1)  # Add channel dimension for Conv1d
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)  # Shape: [batch_size, 256, input_dim / 2]

        # BiLSTM
        x = x.permute(0, 2, 1)  # Reshape for LSTM: [batch_size, seq_len, features]
        lstm_out, _ = self.bilstm(x)  # Shape: [batch_size, input_dim / 2, 256]

        # Flatten for fully connected layer
        x = lstm_out.contiguous().view(lstm_out.size(0), -1)
        shared_features = torch.relu(self.fc_shared(x))  # Shape: [batch_size, 128]

        # Emotional Distress Classification with Attention
        attn_weights_emotional = torch.softmax(self.attention_emotional(shared_features), dim=1)
        emotional_features = attn_weights_emotional * shared_features
        emotional_output = torch.sigmoid(self.fc_emotional(emotional_features))

        # Provoking Violence Classification with Attention
        attn_weights_violence = torch.softmax(self.attention_violence(shared_features), dim=1)
        violence_features = attn_weights_violence * shared_features
        violence_output = torch.sigmoid(self.fc_violence(violence_features))

        # Individual Harassment Classification with Attention
        attn_weights_harassment = torch.softmax(self.attention_harassment(shared_features), dim=1)
        harassment_features = attn_weights_harassment * shared_features
        harassment_output = torch.sigmoid(self.fc_harassment(harassment_features))

        # Concatenate outputs for multilabel classification
        return torch.cat((emotional_output, violence_output, harassment_output), dim=1)

# Instantiate model, loss function, optimizer
input_dim = X.shape[1]  # Number of features in embeddings
output_dim = y_binary.shape[1]  # Number of labels

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridCNNBiLSTMClassifier(input_dim=input_dim, output_dim=output_dim).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].cpu().numpy()

            outputs = model(input_ids).cpu().numpy()
            predictions.append(outputs)
            true_labels.append(labels)

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Epoch 1/10, Loss: 0.1657
Epoch 2/10, Loss: 0.1521
Epoch 3/10, Loss: 0.1493
Epoch 4/10, Loss: 0.1469
Epoch 5/10, Loss: 0.1438
Epoch 6/10, Loss: 0.1399
Epoch 7/10, Loss: 0.1343
Epoch 8/10, Loss: 0.1264
Epoch 9/10, Loss: 0.1176
Epoch 10/10, Loss: 0.1096
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.85      0.92      0.89      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.95      0.98      0.96     30805
            macro avg       0.95      0.97      0.96     30805
         weighted avg       0.95      0.98      0.96     30805
          samples avg       0.95      0.97      0.95     30805

Overall Accuracy: 0.8034


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Load and preprocess the dataset with SBERT embeddings
data = pd.read_csv('embedded_sbert.csv')
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # SBERT Features (already embedded)
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values
y_binary = (y > 0).astype(int)  # Binarize labels for multi-label classification

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Sentence embeddings for SBERT (no sequence dim)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
class MultiLabelClassificationNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(MultiLabelClassificationNet, self).__init__()
        # Define the attention layers
        self.multi_attn1 = nn.MultiheadAttention(embed_dim=input_dim, num_heads=8, batch_first=True)
        self.multi_attn2 = nn.MultiheadAttention(embed_dim=input_dim, num_heads=8, batch_first=True)

        # Fully connected layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        # Reshape input to fit attention layers as batch x 1 x input_dim
        x = x.unsqueeze(1)  # Add a sequence dimension for compatibility

        # Attention layers
        attn_out1, _ = self.multi_attn1(x, x, x)
        attn_out2, _ = self.multi_attn2(attn_out1, attn_out1, attn_out1)

        # Mean pooling across sequence dimension to get fixed-size output
        x = attn_out2.mean(dim=1)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for multi-label classification
        return x

# Hyperparameters
input_dim = 384  # SBERT embeddings are typically 384-dimensional
hidden_dim = 128
num_labels = y_train.shape[1]
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Model, loss function, and optimizer
model = MultiLabelClassificationNet(input_dim=input_dim, hidden_dim=hidden_dim, num_labels=num_labels).to(device)
criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Move tensors to device
X_train_tensor, y_train_tensor = X_train_tensor.to(device), y_train_tensor.to(device)
X_val_tensor, y_val_tensor = X_val_tensor.to(device), y_val_tensor.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    y_val_pred = model(X_val_tensor)
    y_val_pred_binary = (y_val_pred > 0.5).int()

# Convert predictions and labels to numpy for evaluation
y_val_pred_np = y_val_pred_binary.cpu().numpy()
y_val_np = y_val_tensor.cpu().numpy()

# Print classification report
print("Classification Report:")
print(classification_report(y_val_np, y_val_pred_np, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val_np, y_val_pred_np)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, y_pred, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display examples from the validation set
display_examples(X_val, y_val_np, y_val_pred_np, num_examples=10)


Epoch [1/10], Loss: 0.6911
Epoch [2/10], Loss: 0.6790
Epoch [3/10], Loss: 0.6588
Epoch [4/10], Loss: 0.6234
Epoch [5/10], Loss: 0.5513
Epoch [6/10], Loss: 0.4339
Epoch [7/10], Loss: 0.2904
Epoch [8/10], Loss: 0.1917
Epoch [9/10], Loss: 0.1990
Epoch [10/10], Loss: 0.2362
Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.82      1.00      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.93      1.00      0.97     30805
            macro avg       0.93      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.93      0.99      0.96     30805

Overall Accuracy: 0.8200
Example 1:
True Labels: [1. 1. 1.]
Predicted Labels: [1 1 1]

Example 2:
True Labels: [1. 1. 1.]
Predicted Labels: [1 1 1]

Example 3:
True Labels: [1. 1. 1.]
Predicted Label

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
# Load and preprocess the dataset
data = pd.read_csv('embedded_sbert.csv')
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Features
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values
y_binary = (y > 0).astype(int)  # Binarize labels for multi-label classification
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)
# Construct graph using k-NN
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # k-NN with cosine distance
knn.fit(X_train)
distances, indices = knn.kneighbors(X_train)
# Construct edge_index for graph (using k-NN results)
edge_index = []
for i in range(len(indices)):
    for j in indices[i]:
        edge_index.append([i, j])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
# Create a graph data object for PyTorch Geometric
train_data = Data(x=X_train_tensor, edge_index=edge_index, y=y_train_tensor)
# GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_labels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)  # Multi-label classification

# Hyperparameters
input_dim = X_train.shape[1]  # Feature dimension (size of embedded vector)
hidden_dim = 128
num_labels = y_train.shape[1]
num_epochs = 10
learning_rate = 0.001
# Model, loss, and optimizer
model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, num_labels=num_labels)
criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(train_data)

    # Calculate loss
    loss = criterion(output, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")


# Construct graph for validation data using k-NN
knn_val = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_val.fit(X_val)
distances_val, indices_val = knn_val.kneighbors(X_val)
# Construct edge_index for validation data (using k-NN results)
edge_index_val = []
for i in range(len(indices_val)):
    for j in indices_val[i]:
        edge_index_val.append([i, j])
edge_index_val = torch.tensor(edge_index_val, dtype=torch.long).t().contiguous()
# Create validation graph data object
val_data = Data(x=X_val_tensor, edge_index=edge_index_val, y=y_val_tensor)
# Evaluation
model.eval()
with torch.no_grad():
    output = model(val_data)  # Use the validation data
    y_pred_binary_val = (output > 0.5).int()

# Ensure correct shape for the validation set
y_pred_binary_val_np = y_pred_binary_val.cpu().numpy()
y_val_np = y_val_tensor.cpu().numpy()

# Classification Report
print("Classification Report:")
print(classification_report(y_val_np, y_pred_binary_val_np, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Overall accuracy
overall_accuracy = accuracy_score(y_val_np, y_pred_binary_val_np)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Epoch [1/10], Loss: 0.7064
Epoch [2/10], Loss: 0.6869
Epoch [3/10], Loss: 0.6692
Epoch [4/10], Loss: 0.6529
Epoch [5/10], Loss: 0.6378
Epoch [6/10], Loss: 0.6235
Epoch [7/10], Loss: 0.6098
Epoch [8/10], Loss: 0.5964
Epoch [9/10], Loss: 0.5833
Epoch [10/10], Loss: 0.5704
Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.82      1.00      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.93      1.00      0.97     30805
            macro avg       0.93      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.93      0.99      0.96     30805

Overall Accuracy: 0.8195


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import re
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Load the dataset
file_path = '/newalternatedataset.csv'  # Update this path if needed
df = pd.read_csv(file_path)

# Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

# Apply preprocessing to the text column
df['processed_text'] = df['text'].apply(preprocess_text)

# Load GPT-Neo model and tokenizer
model_name = 'EleutherAI/gpt-neo-125M'  # You can choose a larger model if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

# Set the padding token to be the EOS token
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token for padding

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to generate embeddings for a batch of texts
def generate_embeddings_batch(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True) # This line is changed to include output_hidden_states=True
        # Use the hidden states or specific token embeddings for the embedding
        # Access the last hidden state from the hidden_states tuple
        embeddings = outputs.hidden_states[-1].mean(dim=1)  # Average pooling # This line is changed
    return embeddings.cpu().numpy().tolist()  # Move back to CPU and convert to list

# Generate embeddings in batches
batch_size = 8  # Adjust based on GPU memory
embeddings = []
for i in range(0, len(df['processed_text']), batch_size):
    batch_texts = df['processed_text'][i:i + batch_size].tolist()
    batch_embeddings = generate_embeddings_batch(batch_texts)
    embeddings.extend(batch_embeddings)

# Add embeddings to the DataFrame
df['embedding'] = embeddings

# Drop the original text and processed_text columns
df = df.drop(columns=['text', 'processed_text'])

# Save to a new CSV file
df.to_csv('/content/embedded_gptneo.csv', index=False)

print("Embedding complete and saved as 'embedded_gptneo.csv'")


Embedding complete and saved as 'embedded_gptneo.csv'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier

# Load the dataset
data = pd.read_csv('/content/embedded_gptneo.csv')

# Convert the 'embedding' column to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Convert to numpy array

# Assuming the labels are present in the CSV file
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Check shapes of X and y
print(f"Shape of X: {X.shape}")  # Should be (n_samples, n_features)
print(f"Shape of y: {y.shape}")  # Should be (n_samples, n_labels)

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Define an XGBoost classifier with MultiOutputClassifier
model = MultiOutputClassifier(XGBClassifier(
    objective='binary:logistic',
    tree_method='gpu_hist',       # Use GPU acceleration
    eval_metric='logloss',        # Log loss for binary classification
    predictor='gpu_predictor',
    learning_rate=0.01,
    max_depth=6,
    verbosity=1
))

# Fit the model
model.fit(X_train, y_train)

# Predict probabilities for the validation set
y_pred_proba = model.predict_proba(X_val)

# Prepare an array to store binary predictions
y_pred_binary = np.zeros(y_val.shape)

# Extract probabilities and convert them to binary predictions
for i in range(len(y_pred_proba)):  # Iterate over each label's predictions
    # Get the probabilities for the positive class (1) for the i-th label
    y_pred_binary[:, i] = (y_pred_proba[i][:, 1] > 0.5).astype(int)

# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {X[i]}\n")

# Display examples from the validation set
display_examples(y_pred_binary, y_val, num_examples=10)


Shape of X: (54932, 768)
Shape of y: (54932, 3)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:37:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:37:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:37:11] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:37:11] WARNING: /workspace/src/learner.cc:740:

Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.82      1.00      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.93      1.00      0.97     30805
            macro avg       0.93      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.93      0.99      0.96     30805

Overall Accuracy: 0.8201
Example 1:
True Labels: [1 1 1]
Predicted Labels: [1. 1. 1.]

Example 2:
True Labels: [1 1 1]
Predicted Labels: [1. 1. 1.]

Example 3:
True Labels: [1 1 1]
Predicted Labels: [1. 1. 1.]

Example 4:
True Labels: [1 1 1]
Predicted Labels: [1. 1. 1.]

Example 5:
True Labels: [1 1 1]
Predicted Labels: [1. 1. 1.]

Example 6:
True Labels: [1 1 1]
Predicted Labels: [1. 1. 1.]

Example 7:
True Labels: [1 0 1]
Predicted Labels: [1. 1. 1.]

Example 8:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import AdamW

# Load the dataset containing GPT-Neo embeddings
data = pd.read_csv('embedded_gptneo.csv')

# Convert the 'embedding' column to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Create a custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define a transformer-based model for multilabel classification
class TransformerMultilabelClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformerMultilabelClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # First layer
        self.dropout = nn.Dropout(0.3)  # Dropout layer
        self.fc2 = nn.Linear(256, output_dim)  # Second layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for multilabel output
        return x

# Instantiate the model
input_dim = X.shape[1]  # Number of features from GPT-Neo embeddings
output_dim = y_binary.shape[1]  # Number of labels
model = TransformerMultilabelClassifier(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for multilabel
optimizer = AdamW(model.parameters(), lr=0.001)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            predictions.append(outputs.cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=10):
    y_pred = (X > 0.5).astype(int)  # Apply threshold to get binary predictions
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display examples from the validation set
display_examples(y_pred_binary, y_val, num_examples=10)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.1675
Epoch 2/10, Loss: 0.1610
Epoch 3/10, Loss: 0.1591
Epoch 4/10, Loss: 0.1573
Epoch 5/10, Loss: 0.1556
Epoch 6/10, Loss: 0.1544
Epoch 7/10, Loss: 0.1540
Epoch 8/10, Loss: 0.1547
Epoch 9/10, Loss: 0.1538
Epoch 10/10, Loss: 0.1518
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.83      0.99      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.94      1.00      0.97     30805
            macro avg       0.94      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.94      0.99      0.96     30805

Overall Accuracy: 0.8210
Example 1:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 2:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 3:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 4:
True Labels: [1 1 1]
Predicte

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, RepeatVector, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter
from sklearn.metrics import accuracy_score

# Load your dataset containing GPT-Neo embeddings
data = pd.read_csv('embedded_gptneo.csv')

# Extract features and labels
X_embedded = np.array(data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=',')).tolist())  # Convert embeddings to array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values  # Labels

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)  # Presence or absence

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_embedded, y_binary, test_size=0.2, random_state=42)

# Function to calculate inverse frequency weights for each class
def compute_inverse_freq_weights(y):
    class_weights = {}
    for i in range(y.shape[1]):  # Loop over each label
        label_counts = Counter(y[:, i])  # Count occurrences of 0 and 1 in each label
        total = len(y)  # Total number of samples
        # Calculate weight for each class as inverse of its frequency
        class_weights[i] = {
            0: total / (2 * label_counts[0]),  # Weight for class 0 (negative)
            1: total / (2 * label_counts[1])   # Weight for class 1 (positive)
        }
    return class_weights

# Compute inverse frequency weights for the training labels
class_weights = compute_inverse_freq_weights(y_binary)

# Reshape the data for CNN input
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)

# Reshape y for the decoder to match the output
num_time_steps = X_train.shape[1]  # This should match the sequence length in your data
y_train_seq = np.repeat(y_train[:, np.newaxis, :], num_time_steps, axis=1)
y_val_seq = np.repeat(y_val[:, np.newaxis, :], num_time_steps, axis=1)

# Ensure the types are float32
y_train_seq = y_train_seq.astype(np.float32)
y_val_seq = y_val_seq.astype(np.float32)

# Define custom weighted binary crossentropy loss function
def weighted_binary_crossentropy(y_true, y_pred):
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Reshape bce_loss to add a dimension for the output classes
    bce_loss = tf.expand_dims(bce_loss, axis=-1)  # Add dimension to match weights

    # Compute weights for the true labels (0 or 1)
    weights = tf.where(tf.equal(y_true, 1),
                       class_weights_tensor[..., 1],  # Weight for class 1 (positive)
                       class_weights_tensor[..., 0])  # Weight for class 0 (negative)

    # Apply the weights to the binary crossentropy loss
    weighted_loss = bce_loss * weights

    return tf.reduce_mean(weighted_loss)

# Convert the class weights into a tensor for use in the loss function
output_dim = y_binary.shape[1]  # Number of output labels
class_weights_tensor = np.array([[class_weights[i][0], class_weights[i][1]] for i in range(output_dim)], dtype=np.float32)
class_weights_tensor = tf.constant(class_weights_tensor)

# Model creation function
def create_seq2seq_cnn_model(input_shape, output_dim):
    # Encoder
    encoder_inputs = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(encoder_inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=64, kernel_size=2, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # Flatten the output before Dense
    x = Flatten()(x)
    encoder_outputs = Dense(128, activation='relu')(x)

    # Expand dimensions to match the expected input for LSTM
    encoder_outputs = RepeatVector(num_time_steps)(encoder_outputs)

    # Decoder
    decoder_lstm = LSTM(32, return_sequences=True)(encoder_outputs)
    decoder_outputs = TimeDistributed(Dense(output_dim, activation='sigmoid'))(decoder_lstm)

    # Create the model
    model = Model(encoder_inputs, decoder_outputs)
    return model

# Initialize and compile the model
input_shape = (X_embedded.shape[1], 1)  # Input shape
model = create_seq2seq_cnn_model(input_shape, output_dim)
model.compile(optimizer='adam', loss=weighted_binary_crossentropy, metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train_seq,
                    batch_size=32,
                    epochs=10,
                    validation_data=(X_val, y_val_seq),
                    callbacks=[early_stopping])

# Predictions on the validation set
y_pred_probs = model.predict(X_val)

# Set a threshold for binary classification based on probabilities
threshold = 0.5
y_pred = (y_pred_probs[:, -1, :] > threshold).astype(int)  # Get predictions from the last time step

# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy by comparing the exact match of all labels
overall_accuracy = accuracy_score(y_val, y_pred)

# Print the overall accuracy
print(f"\nOverall Accuracy: {overall_accuracy:.4f}")

# Display sample examples from the validation set
def display_examples(X, y_true, y_pred, num_examples=25):
    for i in range(num_examples):
        print(f"Example {i+1}:")
        print(f"True Labels (y): {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display sample examples from the validation set
display_examples(X_val, y_val, y_pred)


Epoch 1/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 202s 116ms/step - accuracy: 0.4425 - loss: 0.5892 - val_accuracy: 0.0028 - val_loss: 0.6920
Epoch 2/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 188s 111ms/step - accuracy: 0.1481 - loss: 0.6146 - val_accuracy: 0.0028 - val_loss: 0.6994
Epoch 3/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 199s 109ms/step - accuracy: 0.3323 - loss: 0.6341 - val_accuracy: 0.0028 - val_loss: 0.7009
Epoch 4/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 148s 107ms/step - accuracy: 0.0427 - loss: 0.6450 - val_accuracy: 0.0028 - val_loss: 0.6957
Epoch 5/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 208s 112ms/step - accuracy: 0.3888 - loss: 0.5985 - val_accuracy: 0.0028 - val_loss: 0.6943
Epoch 6/10
1374/1374 ━━━━━━━━━━━━━━━━━━━━ 197s 109ms/step - accuracy: 0.0129 - loss: 0.6377 - val_accuracy: 0.9932 - val_loss: 0.6956
344/344 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step
Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('embedded_gptneo.csv')

# Convert 'embedded_text' to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Hybrid Model with CNN, BiLSTM, and Separate Attention Layers
class HybridCNNBiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HybridCNNBiLSTMClassifier, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)

        # BiLSTM layer
        self.bilstm = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)

        # Shared fully connected layer
        self.fc_shared = nn.Linear(128 * 2 * (input_dim // 2), 128)  # Adjusted for BiLSTM output

        # Separate attention and classification layers for each label
        self.attention_emotional = nn.Linear(128, 1)
        self.fc_emotional = nn.Linear(128, 1)

        self.attention_violence = nn.Linear(128, 1)
        self.fc_violence = nn.Linear(128, 1)

        self.attention_harassment = nn.Linear(128, 1)
        self.fc_harassment = nn.Linear(128, 1)

    def forward(self, x):
        # CNN
        x = x.unsqueeze(1)  # Add channel dimension for Conv1d
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)  # Shape: [batch_size, 256, input_dim / 2]

        # BiLSTM
        x = x.permute(0, 2, 1)  # Reshape for LSTM: [batch_size, seq_len, features]
        lstm_out, _ = self.bilstm(x)  # Shape: [batch_size, input_dim / 2, 256]

        # Flatten for fully connected layer
        x = lstm_out.contiguous().view(lstm_out.size(0), -1)
        shared_features = torch.relu(self.fc_shared(x))  # Shape: [batch_size, 128]

        # Emotional Distress Classification with Attention
        attn_weights_emotional = torch.softmax(self.attention_emotional(shared_features), dim=1)
        emotional_features = attn_weights_emotional * shared_features
        emotional_output = torch.sigmoid(self.fc_emotional(emotional_features))

        # Provoking Violence Classification with Attention
        attn_weights_violence = torch.softmax(self.attention_violence(shared_features), dim=1)
        violence_features = attn_weights_violence * shared_features
        violence_output = torch.sigmoid(self.fc_violence(violence_features))

        # Individual Harassment Classification with Attention
        attn_weights_harassment = torch.softmax(self.attention_harassment(shared_features), dim=1)
        harassment_features = attn_weights_harassment * shared_features
        harassment_output = torch.sigmoid(self.fc_harassment(harassment_features))

        # Concatenate outputs for multilabel classification
        return torch.cat((emotional_output, violence_output, harassment_output), dim=1)

# Instantiate model, loss function, optimizer
input_dim = X.shape[1]  # Number of features in embeddings
output_dim = y_binary.shape[1]  # Number of labels

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridCNNBiLSTMClassifier(input_dim=input_dim, output_dim=output_dim).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].cpu().numpy()

            outputs = model(input_ids).cpu().numpy()
            predictions.append(outputs)
            true_labels.append(labels)

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Epoch 1/10, Loss: 0.1625
Epoch 2/10, Loss: 0.1521
Epoch 3/10, Loss: 0.1472
Epoch 4/10, Loss: 0.1409
Epoch 5/10, Loss: 0.1324
Epoch 6/10, Loss: 0.1213
Epoch 7/10, Loss: 0.1101
Epoch 8/10, Loss: 0.0999
Epoch 9/10, Loss: 0.0920
Epoch 10/10, Loss: 0.0865
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.85      0.92      0.88      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.95      0.98      0.96     30805
            macro avg       0.94      0.97      0.96     30805
         weighted avg       0.95      0.98      0.96     30805
          samples avg       0.95      0.97      0.95     30805

Overall Accuracy: 0.7974


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Function to make predictions on a given batch of data
def predict_examples(model, examples):
    model.eval()
    with torch.no_grad():
        examples_tensor = torch.tensor(examples, dtype=torch.float32).to(device)
        predictions = model(examples_tensor).cpu().numpy()
    return predictions

# Select a few samples from the validation set for testing
num_examples = 5  # Number of examples to predict
example_data = X_val[:num_examples]
example_labels = y_val[:num_examples]

# Make predictions
predicted_probs = predict_examples(model, example_data)
predicted_labels = (predicted_probs > 0.5).astype(int)  # Binarize the predictions

# Print results
for i in range(num_examples):
    print(f"Example {i+1}")
    print(f"True Labels: {example_labels[i]}")
    print(f"Predicted Probabilities: {predicted_probs[i]}")
    print(f"Predicted Labels: {predicted_labels[i]}")
    print()


Example 1
True Labels: [1 1 1]
Predicted Probabilities: [1.        0.9938705 0.9999999]
Predicted Labels: [1 1 1]

Example 2
True Labels: [1 1 1]
Predicted Probabilities: [0.99999976 0.93573344 1.        ]
Predicted Labels: [1 1 1]

Example 3
True Labels: [1 1 1]
Predicted Probabilities: [0.9999895  0.96975243 0.99255896]
Predicted Labels: [1 1 1]

Example 4
True Labels: [1 1 1]
Predicted Probabilities: [0.9999205  0.71981037 0.99997795]
Predicted Labels: [1 1 1]

Example 5
True Labels: [1 1 1]
Predicted Probabilities: [0.9999995 0.9998004 0.9999956]
Predicted Labels: [1 1 1]



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Load and preprocess the dataset
data = pd.read_csv('embedded_gptneo.csv')
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Features
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values
y_binary = (y > 0).astype(int)  # Binarize labels for multi-label classification

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MultiHeadAttn(nn.Module):
    def __init__(self):
        super().__init__()
        self.multi_attn = nn.MultiheadAttention(embed_dim=768, num_heads=12, batch_first=True)

    def forward(self, q, k, v):
        mv_1, _ = self.multi_attn(q, k, v)
        return mv_1

class MultiLabelClassificationNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(MultiLabelClassificationNet, self).__init__()
        self.multiAttn1 = MultiHeadAttn()
        self.multiAttn2 = MultiHeadAttn()
        self.multiAttn3 = MultiHeadAttn()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        # Apply attention layers
        attn_out1 = self.multiAttn1(x, x, x)
        attn_out2 = self.multiAttn2(attn_out1, attn_out1, attn_out1)
        attn_out3 = self.multiAttn3(attn_out2, attn_out2, attn_out2)

        # Use mean pooling to reduce sequence dimension for fully connected layer
        x = attn_out3.mean(dim=1)  # (batch_size, embed_dim)

        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for multi-label classification
        return x

# Hyperparameters
input_dim = 768  # Example input dimension from embeddings
hidden_dim = 128
num_labels = y_train.shape[1]
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Model, loss, and optimizer
model = MultiLabelClassificationNet(input_dim=input_dim, hidden_dim=hidden_dim, num_labels=num_labels).to(device)
criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Move tensors to device
X_train_tensor, y_train_tensor = X_train_tensor.to(device), y_train_tensor.to(device)
X_val_tensor, y_val_tensor = X_val_tensor.to(device), y_val_tensor.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    y_val_pred = model(X_val_tensor)
    y_val_pred_binary = (y_val_pred > 0.5).int()

# Convert predictions and labels to numpy for evaluation
y_val_pred_np = y_val_pred_binary.cpu().numpy()
y_val_np = y_val_tensor.cpu().numpy()

# Print classification report
print("Classification Report:")
print(classification_report(y_val_np, y_val_pred_np, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val_np, y_val_pred_np)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, y_pred, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display examples from the validation set
display_examples(X_val, y_val_np, y_val_pred_np, num_examples=10)


Epoch [1/10], Loss: 0.6747
Epoch [2/10], Loss: 0.2009
Epoch [3/10], Loss: 1.0054
Epoch [4/10], Loss: 6.2636
Epoch [5/10], Loss: 6.2950
Epoch [6/10], Loss: 6.2922
Epoch [7/10], Loss: 6.2443
Epoch [8/10], Loss: 5.3609
Epoch [9/10], Loss: 1.0930
Epoch [10/10], Loss: 1.7908
Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.82      1.00      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.93      1.00      0.97     30805
            macro avg       0.93      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.93      0.99      0.96     30805

Overall Accuracy: 0.8201
Example 1:
True Labels: [1. 1. 1.]
Predicted Labels: [1 1 1]

Example 2:
True Labels: [1. 1. 1.]
Predicted Labels: [1 1 1]

Example 3:
True Labels: [1. 1. 1.]
Predicted Label

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


In [4]:
# Load and preprocess the dataset
data = pd.read_csv('embedded_gptneo.csv')
data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedding'].tolist())  # Features
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values
y_binary = (y > 0).astype(int)  # Binarize labels for multi-label classification

In [5]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

In [6]:
# Construct graph using k-NN
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # k-NN with cosine distance
knn.fit(X_train)
distances, indices = knn.kneighbors(X_train)


In [7]:
# Construct edge_index for graph (using k-NN results)
edge_index = []
for i in range(len(indices)):
    for j in indices[i]:
        edge_index.append([i, j])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

In [8]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

In [9]:
# Create a graph data object for PyTorch Geometric
train_data = Data(x=X_train_tensor, edge_index=edge_index, y=y_train_tensor)

In [10]:
# GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_labels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)  # Multi-label classification

In [11]:
# Hyperparameters
input_dim = X_train.shape[1]  # Feature dimension (size of embedded vector)
hidden_dim = 128
num_labels = y_train.shape[1]
num_epochs = 10
learning_rate = 0.001

In [12]:
# Model, loss, and optimizer
model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, num_labels=num_labels)
criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(train_data)

    # Calculate loss
    loss = criterion(output, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.2327
Epoch [2/10], Loss: 0.3359
Epoch [3/10], Loss: 0.3304
Epoch [4/10], Loss: 0.3447
Epoch [5/10], Loss: 0.3339
Epoch [6/10], Loss: 0.3920
Epoch [7/10], Loss: 0.3831
Epoch [8/10], Loss: 0.3926
Epoch [9/10], Loss: 0.4202
Epoch [10/10], Loss: 0.4298


In [14]:
# Construct graph for validation data using k-NN
knn_val = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_val.fit(X_val)
distances_val, indices_val = knn_val.kneighbors(X_val)

In [15]:
# Construct edge_index for validation data (using k-NN results)
edge_index_val = []
for i in range(len(indices_val)):
    for j in indices_val[i]:
        edge_index_val.append([i, j])
edge_index_val = torch.tensor(edge_index_val, dtype=torch.long).t().contiguous()

In [16]:
# Create validation graph data object
val_data = Data(x=X_val_tensor, edge_index=edge_index_val, y=y_val_tensor)

In [17]:
# Evaluation
model.eval()
with torch.no_grad():
    output = model(val_data)  # Use the validation data
    y_pred_binary_val = (output > 0.5).int()

# Ensure correct shape for the validation set
y_pred_binary_val_np = y_pred_binary_val.cpu().numpy()
y_val_np = y_val_tensor.cpu().numpy()

In [18]:

# Classification Report
print("Classification Report:")
print(classification_report(y_val_np, y_pred_binary_val_np, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Overall accuracy
overall_accuracy = accuracy_score(y_val_np, y_pred_binary_val_np)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.83      0.95      0.89      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.94      0.99      0.96     30805
            macro avg       0.94      0.98      0.96     30805
         weighted avg       0.94      0.99      0.96     30805
          samples avg       0.94      0.98      0.95     30805

Overall Accuracy: 0.8002


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
